#Auto adjusted Machine Learning models

Machine learning regressors adjusted in a fully automated manner, in order to use them as surrogate models for a CAE VAWT model.

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression
from sklearn.model_selection import GridSearchCV
from joblib import dump, load
import scipy.io
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import pandas as pd 

In [ ]:
def Pythoninit():
    from sklearn.neural_network import MLPRegressor
    from sklearn.svm import SVR
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.linear_model import BayesianRidge, LinearRegression
    from sklearn.model_selection import GridSearchCV
    from joblib import dump, load
    import scipy.io
    import numpy as np
    from sklearn.metrics import r2_score
    from sklearn.metrics import mean_squared_error
    

## Auto adjusting surrogate models

In [ ]:
def AutoSB(inputs_train,outputs_train):
     #Support Vector Regressor    
     model_SVM=SVR()
     param_grid_SVM = { 'kernel':['linear','rbf'],
                   'C':[0.01,0.1,1,10,100],
                   'gamma':[1,10,100,1000,10000,'auto']}
     gridSVM = GridSearchCV(model_SVM,param_grid_SVM,refit=True,verbose=0,scoring='neg_mean_squared_error')
     gridSVM.fit(inputs_train,outputs_train)
     print(gridSVM.best_params_)
     print(gridSVM.best_score_)
    
     #Random Forest Regressor
     model_RFR=RandomForestRegressor()
     param_grid_RFR = { 'n_estimators':[200, 600, 1000, 1400, 1800], #800,1200
                        'max_features':['auto', 'log2'],#'sqrt'
                        'max_depth': [10, 50, 90, None],
                        'min_samples_leaf': [1, 2, 4],
                        'min_samples_split': [2, 5, 10]}#1
    
     gridRFR = GridSearchCV(model_RFR,param_grid_RFR,refit=True,verbose=0,scoring='neg_mean_squared_error')
     gridRFR.fit(inputs_train,outputs_train)
     print(gridRFR.best_params_)
     print(gridRFR.best_score_)
    
     #Bayesian Ridge Regressor
     model_BR= BayesianRidge(n_iter=500)
     param_grid_BR = {'alpha_1':[1e-4,1e-6,1e-7],
                      'alpha_2':[1e-4,1e-6,1e-7],
                      'lambda_1':[1e-4,1e-6,1e-7],
                      'lambda_2':[1e-4,1e-6,1e-7]}
     gridBR= GridSearchCV(model_BR,param_grid_BR,refit=True,verbose=0,scoring='neg_mean_squared_error')
     gridBR.fit(inputs_train,outputs_train)
     print(gridBR.best_params_)
     print(gridBR.best_score_)     
    
    #Multi Layer Perceptron Regressor
    model_MLP = MLPRegressor()
    param_grid_MLP = {'hidden_layer_sizes':[(20),(5,5),(10,10),(15,15),(20,20),(30,30),(10,10,10),(15,15,15),(20,20,20),(30,30,30)],
                       'activation':['tanh','logistic'],
                       'solver':['lbfgs'],                                            
                       'max_iter':[300]}                                                            
    gridMLP = GridSearchCV(model_MLP,param_grid_MLP,refit=True,verbose=0,scoring='neg_mean_squared_error')
    gridMLP.fit(inputs_train,outputs_train)
    print(gridMLP.best_params_)
    print(gridMLP.best_score_)
    

    return  gridSVM,gridRFR,gridBR,gridMLP




In [ ]:
#Loading Training Set
pd_train=pd.read_csv('dataset_DOE_Full_Savonius_Train_Ord.csv')  
Out=pd_train["Cp"].to_numpy()
del pd_train["Cp"]
inputs=pd_train.to_numpy()

In [ ]:
#Fit Models
modeloSVM,ModeloRFR,ModeloBR,ModeloMLP = AutoSB(inputs,Out) 

{'activation': 'tanh', 'hidden_layer_sizes': (30, 30), 'max_iter': 300, 'solver': 'lbfgs'}
-0.000691052208507054


In [ ]:
#Displaying best scores for the training test
print(modeloSVM.best_score_)
print(ModeloRFR.best_score_)
print(ModeloBR.best_score_)
print(ModeloMLP.best_score_)

-9.407737375315907e-05

In [ ]:
#Saving Models
dump(ModeloSVM,'modeloSVM.joblib')
dump(ModeloRFR,'ModeloRFR_3.joblib')
dump(ModeloBR,'ModeloBR.joblib')
dump(ModeloMLP,'ModeloMLP.joblib')

['ModeloMLP.joblib']

In [ ]:
#Loading Models
ModeloSVM = load('modeloSVM.joblib')
ModeloRFR = load('ModeloRFR_2.joblib')
ModeloBR = load('ModeloBR.joblib')
ModeloMLP = load('ModeloMLP.joblib')

In [ ]:
#Displaying best parameters
print(ModeloSVM.best_params_)
print(ModeloRFR.best_params_)
print(ModeloBR.best_params_)
print(ModeloMLP.best_params_)

{'C': 0.01, 'gamma': 1, 'kernel': 'linear'}
{'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1000}
{'alpha_1': 0.0001, 'alpha_2': 1e-07, 'lambda_1': 1e-07, 'lambda_2': 0.0001}
{'activation': 'tanh', 'hidden_layer_sizes': (30, 30), 'max_iter': 300, 'solver': 'lbfgs'}


In [ ]:
#Function to evaluate the surrogate models
def modelEvaluation(moldelList,Experiments,Expout):
    LScore=[];
    MSElist = []
    for model in moldelList:
        MSE = mean_squared_error(Expout, model.predict(Experiments), multioutput='raw_values')
        Score = r2_score(model.predict(Experiments),Expout);
        LScore.append(Score);
        MSElist.append(MSE);
    return LScore,MSElist

In [ ]:
#Displaying R^2 score for the surrogate models using the Training Set
print(r2_score(ModeloSVM.predict(inputs),Out))
print(r2_score(ModeloRFR.predict(inputs),Out))
print(r2_score(ModeloBR.predict(inputs),Out))
print(r2_score(ModeloMLP.predict(inputs),Out))

-4.191568752587685e+30
0.973564017116805
-6.030074666408279
-9.613187483351624


In [ ]:
modelEvaluation([modeloSVM,ModeloRFR,ModeloBR,ModeloMLP],inputs,Out)

([-4.191568752587685e+30,
  0.9814869281746533,
  -6.030074666408279,
  -86.04526353077097],
 [array([0.00080727]),
  array([1.1401727e-05]),
  array([0.00063787]),
  array([0.00079709])])

In [ ]:
#Loading Testing Set
pd_test=pd.read_csv('dataset_DOE_Full_Savonius_Test_Ord.csv')  
Out_test=pd_test["Cp"].to_numpy()
del pd_test["Cp"]
inputs_test=pd_test.to_numpy()

In [ ]:
#Displaying R^2 score for the surrogate models using the Testing Set
print(r2_score(ModeloSVM.predict(inputs_test),Out_test))
print(r2_score(ModeloRFR.predict(inputs_test),Out_test))
print(r2_score(ModeloBR.predict(inputs_test),Out_test))
print(r2_score(ModeloMLP.predict(inputs_test),Out_test))

0.0
0.9818105478108954
-6.73099204782163
-13.636522893638945


In [ ]:
modelEvaluation([modeloSVM,ModeloRFR,ModeloBR,ModeloMLP],inputs_test,Out_test)

([0.0, 0.7517089327699773, -6.73099204782163, -134.49548050163284],
 [array([0.0010106]),
  array([0.00013892]),
  array([0.0009479]),
  array([0.00125735])])